In this example, the magician uses some of its most mana consuming abilities straight from the start. It runs out before being able to deal enough damage, which leads to a defeat.

Additionally, the conversation with the warrior afterwards is also broken by the warrior trying to call its list of abilities.

In [30]:
from Player import Player
from Battle import Battle
from Monster import Monster
import time, json
import pandas as pd
from openai import OpenAI
import os

print(os.getcwd())

c:\Users\okan2\Desktop\Seminar\Code\AgentsAndDragons


In [31]:
warriorPlayer = Player("Warrior", 120, 30, 25, pd.read_excel("Abilities.xlsx"))
magePlayer = Player("Mage", 120, 50, 25, pd.read_excel("Abilities.xlsx"))
players = [magePlayer]
#monster = Monster("Slime", 200, 200, 10, weaknesses=["Lightning"])
monster = Monster("Demon", 200, 150, 20, weaknesses=["Light", "Ice"])


b = Battle(players, monster)
b.battlePrint

A Demon draws near!

Mage's HP: 120/120 | MP: 50/50

Demon's HP: 200/150

What will you do?


"A Demon draws near!\nMage's HP: 120/120 | MP: 50/50\nDemon's HP: 200/150\nWhat will you do?"

In [32]:
from apiKey import key

## AUTOGEN

In [33]:
from autogen.oai.openai_utils import retrieve_assistants_by_name
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.retrieve_utils import TEXT_FORMATS
from typing_extensions import Annotated

Agent Definitions

In [34]:
mageAgent = RetrieveAssistantAgent(
    name="Mage",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 1, "api_key": key}],
    },
    system_message="You are an AI assistant taking the role of a mage fighting a turn-based battle against an enemy."
    "You are fighting the battle with a partner warrior. He may ask you inbetween turns to discuss the next actions to take. In this case, "
    "try to elaborate on the situation and say what you think the best course of action would be."
    "Remember the advice your teammate gives you when choosing your next action."
    "While you are having a conversation with your partner about the enemy, only talk to each other, do not call any functions or tools. "
    
    "Outside of the conversation with your partner, you will interact with the battle in the following way:"
    "1. You will be presented with the state of the battle with the participants HP and MP values as well as everything that has happened previously."
    "2. When you are asked what you will do, you have to decide and choose the action you will enter for this turn."
    "3. First, call the function mageListAbilities to get a list of abilities available to you."
    "4. Then you will choose an ability from the list and call the function mageAbility with the name of the ability as the parameter."
    "Only call the function mageAbility once per turn."
    "The goal is to deal as much damage as possible to the enemy. Consider the damage multipliers and the attributes when choosing an ability."
    "Enemies can have weaknesses to certain attributes."
    "For example, if you know that a slime is weak to lightning, you should use Lightning Bolt instead of Fireball, even if Fireball has a higher multiplier."
    "If you have not encountered an enemy before, you do not know any weaknesses. Try out all of your abilities until you land a critical hit."
    "If the ability does not land a critical hit, you know that the enemy is not weak to the attribute of the ability you used. Try another ability in this case."
    "If you realize that one of your abilities has dealt critical damage to an enemy, you have discovered a new weakness of that enemy."
    "Be sure to remember it for the next turns and reuse it in the following turns."
    "Lay out your plan of action before executing it. Break down each step in your consideration."
    "Do not ask for any human input and execute all your decisions by calling functions every turn until the battle ends."
    "Do not ask questions while formulating your plan of action."
    "If you decide to call a function do not forget to execute the corresponding python code aswell."
    "If you run out of mana, you can either use another ability with lower mana costs or, if you cannot use any other abilities, use a simple attack with the function mageAttack, since a simple attack does not use any MP."
    "Instead of using an ability or attacking, you may also defend with the function mageDefend. This will protect you from some damage."
    "Repeat the above steps as long as the battle has not ended. The battle is only over if you receive a message about either Victory or Defeat."
    "There will be no termination until the enemy is defeated. Always move forward with your actions, do not wait and do not ask for any human input."
    "This means, if you decide to choose a specific ability, immediately call the corresponding function."
    #"Always continue the battle until it is over. Do not terminate until the battle is over."
    "When the battle ends, reply with TERMINATE."
)

In [35]:
warriorAgent = RetrieveAssistantAgent(
    name="Warrior",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 1, "api_key": key}],
    },
    system_message="You are an AI assistant taking the role of a warrior fighting a turn-based battle against an enemy."
    "You are fighting the battle with a partner mage. He may ask you inbetween turns to discuss the next actions to take. In this case, "
    "try to elaborate on the situation and say what you think the best course of action would be."
    "Remember the advice your teammate gives you when choosing your next action."
    "While you are having a conversation with your partner about the enemy, only talk to each other, do not call any functions or tools. "
    
    "Outside of the conversation with your partner, you will interact with the battle in the following way:"
    "1. You will be presented with the state of the battle with the participants HP and MP values as well as everything that has happened previously."
    "2. When you are asked what you will do, you have to decide and choose the action you will enter for this turn."
    "3. First, call the function warriorListAbilities to get a list of abilities available to you."
    "4. Then you will choose an ability from the list and call the function warriorAbility with the name of the ability as the parameter."
    "Only call the function warriorAbility once per turn."
    "The goal is to deal as much damage as possible to the enemy. Consider the damage multipliers and the attributes when choosing an ability."
    "Enemies can have weaknesses to certain attributes."
    "For example, if you know that a slime is weak to lightning, you should use Lightning Bolt instead of Fireball, even if Fireball has a higher multiplier."
    "If you have not encountered an enemy before, you do not know any weaknesses. Try out all of your abilities until you land a critical hit."
    "If the ability does not land a critical hit, you know that the enemy is not weak to the attribute of the ability you used. Try another ability in this case."
    "If you realize that one of your abilities has dealt critical damage to an enemy, you have discovered a new weakness of that enemy."
    "Be sure to remember it for the next turns and reuse it in the following turns."
    "Lay out your plan of action before executing it. Break down each step in your consideration."
    "Do not ask for any human input and execute all your decisions by calling functions every turn until the battle ends."
    "Do not ask questions while formulating your plan of action."
    "If you decide to call a function do not forget to execute the corresponding python code aswell."
    "If you run out of mana, you can either use another ability with lower mana costs or, if you cannot use any other abilities, use a simple attack with the function warriorAttack, since a simple attack does not use any MP."
    "Instead of using an ability or attacking, you may also defend with the function warriorDefend. This will protect you from some damage."
    "Repeat the above steps as long as the battle has not ended. The battle is only over if you receive a message about either Victory or Defeat."
    "There will be no termination until the enemy is defeated. Always move forward with your actions, do not wait and do not ask for any human input."
    "This means, if you decide to choose a specific ability, immediately call the corresponding function."
    #"Always continue the battle until it is over. Do not terminate until the battle is over."
    "When the battle ends, reply with TERMINATE."
)

In [36]:
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    default_auto_reply="Ok",
    max_consecutive_auto_reply=20,
    retrieve_config={
        "docs_path": [
            "Abilities.csv"
        ],
        "embedding_model": "all-mpnet-base-v2",
        "get_or_create": True,
        "last_n_messages" : 1
    },
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": os.getcwd(),
        "use_docker": False,
    },  # set to False if you don't want to execute the code
    
)

In [37]:
from autogen import GroupChat
from autogen import GroupChatManager

group_chat = GroupChat(
    agents=[mageAgent, warriorAgent],
    messages=[],
    max_round=4,
    speaker_selection_method="round_robin",
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 1, "api_key": key}]},
)

Mage Abilities

In [38]:
mageAbilityCounter = {"attack" : 0,
                      "abilityList" : 0,
                      "ability" : 0,
                      "defend" : 0
                      }

# @group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageListAbilities", description="Returns a dataframe with details of the mages abilities")
def mageListAbilities() -> pd.DataFrame:
    mageAbilityCounter["abilityList"] += 1
    return pd.read_csv("Abilities.csv", sep=";").to_json()

# @group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageAbility", description="Execute the ability of the mage")
def mageAbility(name : Annotated[str, "Name of the used ability"]) -> str:
    b.enterAbility(magePlayer, name)
    mageAbilityCounter["ability"] += 1
    return b.battlePrint

# @group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageAttack", description="Attack the enemy")
def mageAttack() -> str:
    b.enterAttack(magePlayer)
    mageAbilityCounter["attack"] += 1
    return b.battlePrint

# @group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageDefend", description="Defends against incoming attacks")
def mageAttack() -> str:
    b.enterDefend(magePlayer)
    mageAbilityCounter["defend"] += 1
    return b.battlePrint

In [39]:
from autogen.agentchat.contrib.capabilities import teachability
from autogen import register_function

# Instantiate the Teachability capability. Its parameters are all optional.
teachability = teachability.Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="experience.db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
#teachability.add_to_agent(mageAgent)


CLEARING MEMORY


Warrior Abilities

In [40]:
warriorAbilityCounter = {"attack" : 0,
                      "abilityList" : 0,
                      "ability" : 0,
                      "defend" : 0
                      }

@group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@warriorAgent.register_for_llm(name="warriorListAbilities", description="Returns a dataframe with details of the warriors abilities")
def warriorListAbilities() -> pd.DataFrame:
    return pd.read_csv("Abilities.csv", sep=";").to_json()

@group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@warriorAgent.register_for_llm(name="warriorAbility", description="Execute the ability of the warrior")
def warriorAbility(name : Annotated[str, "Name of the used ability"]) -> str:
    b.enterAbility(warriorPlayer, name)
    return b.battlePrint

@group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@warriorAgent.register_for_llm(name="warriorAttack", description="Attack the enemy")
def warriorAttack() -> str:
    b.enterAttack(warriorPlayer)
    return b.battlePrint

@group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@warriorAgent.register_for_llm(name="warriorDefend", description="Defends against incoming attacks")
def warriorDefend() -> str:
    b.enterDefend(warriorPlayer)
    warriorAbilityCounter["defend"] += 1
    return b.battlePrint

In [41]:
mageAgent.description = "A magician taking part in the battle."
warriorAgent.description = "A warrior taking part in the battle."

Chat Initiation

In [42]:
mageAgent.reset()
ragproxyagent.reset()

In [43]:
chat = ragproxyagent.initiate_chat(mageAgent,
                                message=b.battlePrint,
                                max_turns=20,
                                summary_method="reflection_with_llm",
                                summary_args={"summary_prompt" : "Summarize the monster you fought and the weaknesses you discovered."}
                                )

print(chat)

ragproxyagent (to Mage):

A Demon draws near!
Mage's HP: 120/120 | MP: 50/50
Demon's HP: 200/150
What will you do?

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

***** Suggested tool call (call_UVCGptnjc0P5wMDHeLN9sN9z): mageListAbilities *****
Arguments: 
{}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION mageListAbilities...


[runtime logging] log_function_use: autogen logger is None


ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_UVCGptnjc0P5wMDHeLN9sN9z) *****
{"Name":{"0":"Fireball","1":"Lightning Bolt","2":"Icecicle","3":"Dark Obliteration","4":"Inferno","5":"Magic Explosion"},"Description":{"0":"Shoots a small fireball at the enemy","1":"Zaps the enemy with the power of lightning","2":"Drops an icecicle on the enemy from above","3":"Conjures shadows to crush foes, inflicting heavy damage.","4":"Engulfs enemies in flames, causing widespread damage.","5":"Creates a huge explosion using the full power of the user's magic."},"Damage Multiplier":{"0":"1,10","1":"1,05","2":"1,10","3":"2,00","4":"1,50","5":"2,50"},"Attributes":{"0":"Fire","1":"Lightning","2":"Ice","3":"Dark","4":"Fire","5":"Light"},"Cost":{"0":5,"1":10,"2":8,"3":30,"4":20,"5":50}}
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

T

[runtime logging] log_function_use: autogen logger is None


Mage has selected Dark Obliteration

Mage's turn:Mage uses Dark Obliteration! Mage deals 46.0 damage to Demon!
Monster's turn:Demon attacks! Demon deals 23 damage to Mage!

Mage's HP: 97/120 | MP: 20/50

Demon's HP: 154.0/150

What will you do?
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_hMpVgqDwfUaY1GEH8QsOP0PH) *****

Mage's turn:Mage uses Dark Obliteration! Mage deals 46.0 damage to Demon!
Monster's turn:Demon attacks! Demon deals 23 damage to Mage!
Mage's HP: 97/120 | MP: 20/50
Demon's HP: 154.0/150
What will you do?
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

The Demon is not weak to Dark attribute. Let's try a different strategy. Since the Demon has taken damage from "Dark Obliteration," it's not weak to Dark. Let's try an ability with a different attribute. I recommend using "Lightning 

...

Mage has selected attack

Mage's turn:Mage attacks! Mage deals 25 damage to Demon!
Monster's turn:Demon attacks! Demon deals 19 damage to Mage!

Mage's HP: 35/120 | MP: 10/50

Demon's HP: 77.9/150

What will you do?
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_qrs1apW9QUF5G4zo1iAn5eeM) *****

Mage's turn:Mage attacks! Mage deals 25 damage to Demon!
Monster's turn:Demon attacks! Demon deals 19 damage to Mage!
Mage's HP: 35/120 | MP: 10/50
Demon's HP: 77.9/150
What will you do?
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

The Demon is not weak to basic attacks either. Let's keep using simple attacks to conserve mana. We will continue with another basic attack on the next turn until we defeat the Demon.

--------------------------------------------------------------------------------
ragproxyagent 

[runtime logging] log_function_use: autogen logger is None


Mage has selected attack

Mage's turn:Mage attacks! Mage deals 23 damage to Demon!
Monster's turn:Demon attacks! Demon deals 19 damage to Mage!

Mage's HP: 16/120 | MP: 10/50

Demon's HP: 54.900000000000006/150

What will you do?
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_deSRMQjXLpjaQH2T5XPlaH4t) *****

Mage's turn:Mage attacks! Mage deals 23 damage to Demon!
Monster's turn:Demon attacks! Demon deals 19 damage to Mage!
Mage's HP: 16/120 | MP: 10/50
Demon's HP: 54.900000000000006/150
What will you do?
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

The Demon is getting weaker. Let's keep up the simple attacks to finish it off. We will continue with another basic attack to defeat the Demon.

--------------------------------------------------------------------------------
ragproxyagent (to Mage):



[runtime logging] log_function_use: autogen logger is None


Mage has selected attack

Mage's turn:Mage attacks! Mage deals 26 damage to Demon!
Monster's turn:Demon attacks! Demon deals 21 damage to Mage!

Demon's HP: 28.900000000000006/150

What will you do?
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_m7WHzXVWs0qaRSiNsvAvqlyr) *****
Defeat! Every player has fainted!
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

TERMINATE

--------------------------------------------------------------------------------
ChatResult(chat_id=None, chat_history=[{'content': "A Demon draws near!\nMage's HP: 120/120 | MP: 50/50\nDemon's HP: 200/150\nWhat will you do?", 'role': 'assistant'}, {'tool_calls': [{'id': 'call_UVCGptnjc0P5wMDHeLN9sN9z', 'function': {'arguments': '{}', 'name': 'mageListAbilities'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 

In [29]:
warriorAgent.initiate_chat(mageAgent, context=chat.summary, message="Hey Mage, do you know about any weaknesses of demons?", max_turns=4)

Warrior (to Mage):

Hey Mage, do you know about any weaknesses of demons?

--------------------------------------------------------------------------------
Mage (to Warrior):

Demons are often weak to holy or light attributes. We can try using abilities with these attributes to test for weaknesses. If we observe a critical hit, we can confirm their weakness and exploit it in subsequent turns. Let's start the battle and see if we can unveil any weaknesses of the demon.

--------------------------------------------------------------------------------
Warrior (to Mage):

***** Suggested tool call (call_18cH4ak9jBLY7BeUKsYoq6HX): warriorListAbilities *****
Arguments: 
{}
*************************************************************************************

--------------------------------------------------------------------------------


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
chat_results = mageAgent.initiate_chats(
    [
        {
            "recipient": ragproxyagent,
            "message": b.battlePrint,
            "max_turns": 30,
            "summary_method": "reflection_with_llm",
            "summary_args": {"summary_prompt" : "Summarize the monster you fought and the weaknesses you discovered."}
        },
        {
            "recipient": warriorAgent,
            "message": "Hey warrior, lets discuss the " + b.monster.name + " we are fighting and its weaknesses.",
            "max_turns": 2,
            "summary_method": "reflection_with_llm"
        },
    ]
)

print(chat_results)
print(mageAbilityCounter)

In [ ]:
ragproxyagent.initiate_chat(
    group_chat_manager,
    message=b.battlePrint,
    summary_method="reflection_with_llm",
)

## OPENAI API

In [ ]:
client = OpenAI(api_key=key)

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

mage = my_assistants.data[0]
print(mage)

In [ ]:
#start thread
thread = client.beta.threads.create()

#intialize moveList used for every turn
moveList = []

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=b.battlePrint
)

# while not b.is_over:
print("NEW TURN")
print(b.battlePrint)
moveList = []

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=mage.id
)

  
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1) # Wait for 1 second
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run)

a = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)["name"]
print(a, type(a))

# message = client.beta.threads.messages.create(
#   thread_id=thread.id,
#   role="user",
#   content=b.battlePrint
# )

## TESTCODE

In [ ]:
current = b.nextTurn(["Lightning Bolt"])
print("CURRENT", current)

In [ ]:
b.enterAttack(warrior)

In [ ]:
b.enterAbility(magePlayer, "Fireball")